# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob, types
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter, PercentFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch, Arc
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','ReverseTreadmill.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

plotting group double errorbars for speed 

In [ ]:
def plot_event_1on1(root, ax, Profiles, colorCode, badAnimals=None, TaskParamToPlot='% good trials',
                   x_pos=None, nInit=slice(0,3), nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(3,6),
                    seed=1, animal_plot=True):
    if badAnimals is None:
        badAnimals=[]
    
    if x_pos is None:
        diff=.35
        x_c=ax.get_xlim()[1]
        x_pos=(x_c-3*diff,x_c-diff,x_c+diff,x_c+3*diff)
    diff=x_pos[1]-x_pos[0]
    
    #getting the data
    animalList,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    
    data=np.empty((len(animalList),4))
    for i,animal in enumerate(animalList):
        preSession,postSession=sessionDict[animal][0], sessionDict[animal][1]
        out=animal_learning_stats(root, animal, PerfParam=TaskParamToPlot,
                                  goodSessions=[*preSession,*postSession],redo=False)
        
        data[i,0]=np.nanmedian(out[:len(preSession)][nInit])
        data[i,1]=np.nanmedian(out[:len(preSession)][nPre])
        data[i,2]=np.nanmedian(out[len(preSession):][nPost])
        data[i,3]=np.nanmedian(out[len(preSession):][nFin])
        
            
    
    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='-o', zorder=5, ms=3, elinewidth=1, color='k')
    
    #plotting individual animals
    if animal_plot:
        np.random.seed(seed=seed)
        _coeff=5000
        for i in range(data.shape[0]):
            if np.any(np.isnan(data[i,:])):
                continue
            jitInit=np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPre =np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[2]-diff/_coeff, high=x_pos[2]+diff/_coeff, size=1)
            jitFin =np.random.uniform(low=x_pos[3]-diff/_coeff, high=x_pos[3]+diff/_coeff, size=1)
            
#             ax.scatter([jitPre,jitFin],data[i,[1,-1]], s=2, c=c, marker='o', edgecolors='none', alpha=.8)
            ax.plot([jitInit,jitPre,jitPost,jitFin],data[i,:], c='gray', lw=.2, alpha=.4)
#             ax.plot([jitPre,jitFin],data[i,[1,-1]], c=c, lw=.4, alpha=.6)

    ax.set_xlim([x_pos[0]-diff/2,x_pos[-1]+diff/2])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Naive','Before','Acute','Stable'])
    ax.xaxis.set_tick_params(rotation=45)
    ax.spines['bottom'].set_bounds(x_pos[0],x_pos[-1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylabel(TaskParamToPlot)

    return data,animalList

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse','Reverse-AfterBreak']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse-Late_DLS']
             }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot='median entrance time (sec)'
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    ax.set_xticks([])
    a=plot_event_1on1(root, ax, Profiles, colorCode={'':'xkcd:aqua','DLS':'xkcd:aqua'}, badAnimals=None, TaskParamToPlot=TaskParamToPlot,
                       x_pos=None, nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(7,10),
                        seed=1, animal_plot=True)

---

plot consecutive trials

In [ ]:
def _anti_routine_trial_numbers(data, xFront=60):
    data.position_correction()
    detect_trial_end(data,trials=data.position.keys())
    
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)

    seqTrial=[]
    for trial,pos in data.position.items():
        #starting from the front
        if pos[trialStart]<xFront and pos[0]<xFront:
            continue

        #not completing the trial
        if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
            continue

        #The trial IS indeed sequential
        seqTrial.append(trial)

    return seqTrial

def plot_consecutive_trajectories(root, session, trials, ax):
    delta=1 #last delta second of position is not recorded
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    data.position_correction()
    try:
        detect_trial_end(data, trials)
    except:
        pass
    fps=data.cameraSamplingRate
    antiR=_anti_routine_trial_numbers(data)

    maxT=0
    for trial in trials:
        time=data.rawTime[trial]+maxT
        maxT=time[-1]+delta
        #plotting position
        ax.plot(time,data.position[trial],'k');
        #plotting entrance time
        ET=data.entranceTime[trial]+data.cameraToTreadmillDelay+time[0]
        if data.entranceTime[trial]!=data.maxTrialDuration[trial]:
            ax.plot(ET,data.position[trial][time[time<ET].argmax()+1],'rx')
        #plotting the goal time
        GT=data.goalTime[trial]+data.cameraToTreadmillDelay+time[0]
        ax.plot(GT,data.position[trial][time[time<GT].argmax()], marker='o',markeredgecolor='r', markerfacecolor='None')
        #plotting the highlight for trials
        x=(time[0], time[0]+data.timeEndTrial[trial]+data.cameraToTreadmillDelay)
        c='xkcd:green' if trial in antiR else 'salmon'
        ax.fill_betweenx(y=(Y1,Y2),x1=x[0],x2=x[1], facecolor=c, edgecolor=None, alpha=.2)
    
    ax.set_xlim([-1,time[-1]+1])
    ax.set_ylim([-2,92])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'','','','','','','','',90])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (cm)', labelpad=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.spines['bottom'].set_bounds(0,time[-1])
    
    return ax


def plot_many_consecutive_trials(root, session, trials, ax):
    delta=1 #last delta second of position is not recorded
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    data.position_correction()
    try:
        detect_trial_end(data, trials)
    except:
        pass
    fps=data.cameraSamplingRate
    antiR=_anti_routine_trial_numbers(data)

    maxT=0
    for trial in trials:
        time=data.rawTime[trial]+maxT
        maxT=time[-1]+delta
        #plotting position
        ax.plot(time,data.position[trial],'k');
    
    ax.set_xlim([-1,time[-1]+1])
    ax.set_ylim([-2,92])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'','','','','','','','',90])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (cm)', labelpad=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.spines['bottom'].set_bounds(0,time[-1])
    ax.set_xticks(np.arange(0,time[-1],10),minor=True)
    
    return ax

    
def add_legend_for_consecutive_trajectories(ax):
    o_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='r', marker='o',
                                       markerfacecolor='None', markersize=6, label='Goal time')

    x_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='r', marker='x',
                                       markerfacecolor='None', markersize=6, label='ET')


    g_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='xkcd:green', label='Correct trial', alpha=1)
    r_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='salmon', label='Error trial', alpha=1)
#     b_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='cyan', label='Omission trial', alpha=1)
    w_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='white', label='Intertrial', alpha=1)



    leg=ax.legend(handles=[g_patch,r_patch,w_patch,o_marker,x_marker],loc=(.85,.1),#mode='expand',
                  ncol=1, fontsize='xx-small', framealpha=.6,frameon=True)
    return leg

In [ ]:
if "__file__" not in dir():
    session='Rat365_2019_08_28_09_27'
#     session='Rat365_2019_09_25_10_15'
    i=1
    trials=range(i,i+100)

    plt.close('all')
    ax=plt.figure(figsize=(20,3)).add_subplot(211);

#     plot_consecutive_trajectories(root, session, trials, ax)
    plot_many_consecutive_trials(root, session, trials, ax)

---

plot single session trajectories

In [ ]:
def plot_session_median_trajectory(data,ax):
    posDict=data.position
    maxL=np.nanmax(list(data.stopFrame.values()))
    maxL=int(maxL)
    position=np.ones((maxL,len(posDict.keys())))*np.nan
    time=np.arange(-data.cameraToTreadmillDelay,
                   (maxL-data.cameraSamplingRate)/data.cameraSamplingRate,
                   1/data.cameraSamplingRate)
    
    
    for i,trial in enumerate(posDict):
        pos=posDict[trial][:data.stopFrame[trial]]
        position[:len(pos),i]=pos
    
    #keeping data where 70% of points exist
    nanSum=np.sum(np.isnan(position),axis=1)
    try:
        maxTraj=np.where(nanSum>.3*position.shape[1])[0][0]
    except IndexError:
        maxTraj=position.shape[1]
    
    
    ax.plot(time[:maxTraj], np.nanmedian(position,axis=1)[:maxTraj], color='navy', lw=2)    

def plot_trajectories(data,ax):
    posDict=data.position
    time=data.timeTreadmill #align on camera
    
    antiR=_anti_routine_trial_numbers(data)
    
    Colors=[]
    for trial in posDict:
        color="xkcd:green"
        alpha=1
        zorder=3
        if trial not in antiR:
            color="salmon"
            alpha=.6
            zorder=1
        Colors.append(color)
        ax.plot(time[trial][:data.stopFrame[trial]], posDict[trial][:data.stopFrame[trial]],
               color=color, lw=.5, alpha=alpha, zorder=zorder)
            
#     ax.fill_betweenx(y=(0,90),x1=0,x2=7, facecolor='gray', edgecolor=None, alpha=.4)
    
    return np.array(Colors)



def plot_trajectories_and_distributions(root, ax, session):
    data=Data(root,session[:6],session,redoPreprocess=False)
    
    color=plot_trajectories(data,ax=ax)
    
    position=get_positions_array_beginning(data,onlyGood=False,raw=False)
    position=position.T
    
#     plot_session_median_trajectory(data,ax)
    
#     props={'color':'k', 'linewidth':1}
#     ax.boxplot(x=data.entranceTime,whis=[5,95],vert=False,
#                positions=[5], widths=5,
#                showcaps=False, showfliers=False,
#                medianprops=props, boxprops=props, whiskerprops=props, zorder=5
#               )
    
    ax.set_xlim([-1.2,11.2])
    ax.set_ylim([-2,90])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(bottom=False, top=False, left=False, right=False,
                  labelbottom=False, labeltop=False, labelleft=False, labelright=False)

In [ ]:
if "__file__" not in dir():
    session='Rat366_2019_10_02_10_50'

    plt.close('all')
    ax=plt.figure(figsize=(3,3)).add_subplot(111);

    plot_trajectories_and_distributions(root, ax, session)

---

AntiSequential trials: Starting from Back for the Reverse Treadmill

In [ ]:
def _anti_sequential_trials(data, xFront=60):
    data.position_correction()
    detect_trial_end(data,trials=data.position.keys())
    
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)

    seqTrial=[]
    for trial,pos in data.position.items():
        #starting from the front
        if pos[trialStart]<xFront and pos[0]<xFront:
            continue

        #not completing the trial
        if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
            continue

        #The trial IS indeed sequential
        seqTrial.append(trial)

    return len(seqTrial)/data.nTrial

In [ ]:
if "__file__" not in dir():
    session='Rat366_2019_10_02_10_50'
    data=Data(root,session[:6],session,redoPreprocess=False)
    
    print(_anti_sequential_trials(data))

---

plot the underlined text

In [ ]:
def add_underlined_text_centered_at(fig,p1,p2, s):
    
    
    fig.add_artist(mpl.lines.Line2D([p1[0],p2[0]] , [p1[1],p2[1]],
                                    lw=1,color='k'))
    
    fig.add_artist(mpl.text.Text(x=np.mean([p1[0],p2[0]]), y=np.mean([p1[1],p2[1]]),
                                        text=s, c='k',
                                        ha='center',va='bottom',fontsize='x-small'))

In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)

    
    add_underlined_text_centered_at(fig,[.2,.5],[.7,.5],'test')
    
    
    plt.show()

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:red'
    DMSColor='xkcd:blue'
    DSColor='xkcd:magenta'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
       

        
    
    #===============================================
    
    # GRID 1 PARAMS
    
    session1='Rat366_2019_10_02_10_50'
    trialsRange1=range(31,40)
    
    
    #===============================================
    
    # GRID 3 PARAMS

    session3Naive='Rat372_2019_06_11_14_56'
    session3Before='Rat372_2019_10_04_16_41'
    session3After ='Rat372_2019_10_21_13_56'
    session3Final ='Rat372_2019_10_31_10_29'
    

    
    
    #===============================================
    
    # GRID 4 PARAMs
    
    profilePre4={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['10','8','6'],
             'Tag':['Reverse','Reverse-AfterBreak']
             }
    profilePost4={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['10','8','6'],
             'Tag':['Reverse-Late_DLS']
             }

    TaskParamToPlot4='median entrance time (sec)'
    color4='xkcd:crimson'
    
    Profiles4=(profilePre4,profilePost4,)
    initSlice4=slice(0,2)
    preSlice4 =slice(-2,None)
    postSlice4=slice(0,2)
    finSlice4 =slice(8,10)
    

    #===============================================
    
    # GRID 5 PARAMs
    
    TaskParamToPlot5 = _anti_sequential_trials.__name__
    

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(6,2)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        
    
    ##########################################
    # 1: plot consecutive trials for Reverse Treadmill
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=.75, right=1, top=1)
    ax1= fig.add_subplot(gs1[0])

    plot_consecutive_trajectories(root, session1, trialsRange1, ax1)
    ax1.set_xticks(np.arange(0,ax1.get_xlim()[-1],5),minor=True)
    
    

    ##########################################
    # 3: plot single session trajectories
    gs3= fig.add_gridspec(nrows=1, ncols=4, wspace=.1, left=0.04, bottom=0.1, right=.49, top=.4)
    ax3_= fig.add_subplot(gs3[0])
    ax3L= fig.add_subplot(gs3[1])
    ax3M= fig.add_subplot(gs3[2])
    ax3R= fig.add_subplot(gs3[3])
    
    plot_trajectories_and_distributions(root, ax3_, session3Naive)
    plot_trajectories_and_distributions(root, ax3L, session3Before)
    plot_trajectories_and_distributions(root, ax3M, session3After)
    plot_trajectories_and_distributions(root, ax3R, session3Final)

    ax3_.set_title('Naive',fontsize='xx-small')
    ax3L.set_title('(#$-1$)',fontsize='xx-small')
    ax3M.set_title('(#$+1$)',fontsize='xx-small')
    ax3R.set_title('(#$+9$)',fontsize='xx-small')
    
    ax3_.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
    ax3_.set_xticks([0,1,2,3,4,5,6,7,8,9,10])
    ax3_.set_xticklabels([0,'','','','','','',7,'','',10])
    ax3_.set_yticks(range(0,91,10))
    ax3_.set_yticklabels([0,'','','','','','','','',90])
    ax3_.spines['bottom'].set_bounds(0,10)
    ax3_.set_xlabel('Trial time (s)',labelpad=0)
    ax3_.set_ylabel('Position (cm)',labelpad=0)
    ax3_.spines['left'].set_bounds(0,90)   
    ax3_.spines['bottom'].set_visible(True)
    ax3_.spines['left'].set_visible(True)

    #adding the Before and After text and line
    y_top=gs3.top+.08
    add_underlined_text_centered_at(fig,p1=(ax3_.get_position().bounds[0],y_top),p2=(ax3L.get_position().bounds[0]+ax3_.get_position().bounds[2],y_top), s='Before')
    add_underlined_text_centered_at(fig,p1=(ax3M.get_position().bounds[0],y_top),p2=(gs3.right,y_top), s='After')
    
    #add text and line for LESION
    x_=np.mean([gs3.right,gs3.left])
    fig.add_artist(mpl.lines.Line2D([x_,x_] , [gs3.bottom,y_top],
                                    linestyle=':',lw=.7,color='gray', zorder=1))
   
    fig.add_artist(mpl.text.Text(x=x_, y=y_top,
                                        text=' Lesion', c='k',
                                        ha='center',va='bottom',fontsize='xx-small',rotation=90))

    

    ##########################################
    # 4: reverse lesion event plot ET
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=.59, bottom=0.1, right=0.74, top=.5)
    ax4= fig.add_subplot(gs4[0])
    
    data4,animalList4=plot_event_1on1(root, ax4, Profiles4, colorCode={'':color4,'DLS':color4}, badAnimals=None, x_pos=(1,2,3,4),
                         TaskParamToPlot=TaskParamToPlot4, nInit=initSlice4, nPre=preSlice4, nPost=postSlice4,nFin=finSlice4)

    
    ax4.plot((1,2,3,4),data4[animalList4.index(session3Naive[:6]),:],'v',
             ms=2,mfc='white',mec='k',markeredgewidth=.5,zorder=50)
    
    ax4.text(1,0,f'$n={len(animalList4)}$ rats ',
          ha='right',va='bottom',fontsize='xx-small',transform=ax4.transAxes)

    ax4.axhline(y=7, ls=':',c='gray',lw=1, zorder=-5)
    ax4.set_ylim([1,8])
    ax4.spines['left'].set_bounds(1,8)
    ax4.set_yticks(np.arange(1,9))
    ax4.set_yticklabels([1,'','',4,'','',7,8])
    ax4.set_ylabel('Entrance time (s)')

    
    
    ##########################################
    # 5: reverse lesion event plot ANTI SEQ
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=.81, bottom=0.1, right=.96, top=.5)
    ax5= fig.add_subplot(gs5[0])
    
    data5,animalList5=plot_event_1on1(root, ax5, Profiles4, colorCode={'':color4,'DLS':color4}, badAnimals=None, x_pos=(1,2,3,4),
                         TaskParamToPlot=TaskParamToPlot5, nInit=initSlice4, nPre=preSlice4, nPost=postSlice4,nFin=finSlice4)

    ax5.plot((1,2,3,4),data5[animalList5.index(session3Naive[:6]),:],'v',
             ms=2,mfc='white',mec='k',markeredgewidth=.5,zorder=50)

    
    ax5.set_ylim([0.2,1])
    ax5.spines['left'].set_bounds(.2,1)
    ax5.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1,decimals=None,symbol=None))
    ax5.set_yticks(np.arange(.2,1.01,.2))
    ax5.set_ylabel('%Anti Routine',labelpad=0)

    
    


    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax1,ax3_,ax4,ax5)
    OFFX=np.array([.07]*len(AXES))
    OFFY=np.array([.03]*len(AXES))
    OFFX[0]=.02
    OFFY[0]=.05
    OFFY[1]=.1
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','ReverseTreadmill.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()